In [ ]:
# =====================================================
# FAKE NEWS DETECTION – DistilBERT + PEFT (LoRA) + WELFAKE
# =====================================================

# 1. INSTALL
!pip install -q transformers datasets peft accelerate bitsandbytes scikit-learn pandas numpy psutil

# 2. IMPORT
import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType
from google.colab import drive
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

# 3. GPU INFO
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
else:
    device_name = "CPU"
    vram_gb = 0.0

print(f"Device: {device_name} | CUDA: {torch.cuda.is_available()} | VRAM: {vram_gb:.1f} GB")

# 4. MOUNT GOOGLE DRIVE
drive.mount('/content/drive', force_remount=False)

def check_drive_space(path="/content/drive/MyDrive"):
    usage = psutil.disk_usage(path)
    print(f"Drive: {usage.used/1e9:.1f}GB / {usage.total/1e9:.1f}GB ({usage.percent}%)")

check_drive_space()

# 5. OUTPUT DIR
OUTPUT_DIR = "/content/drive/MyDrive/WELFake_DistilBERT_LoRA"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 6. LOAD DATA
dataset = load_dataset("davanstrien/WELFake")
df = pd.DataFrame(dataset["train"])

df["content"] = df.get("title","").fillna("") + " [SEP] " + df.get("text","").fillna("")

def clean_text(t):
    if not isinstance(t, str): return ""
    t = t.lower()
    t = re.sub(r'https?://\S+', ' ', t)
    t = re.sub(r'<.*?>', ' ', t)
    t = re.sub(r'[^a-z0-9\s]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

df["content"] = df["content"].apply(clean_text)
df = df[df["content"].str.len() > 20].drop_duplicates(subset=["content"])
print("Dataset cleaned:", len(df))

# 7. SPLIT DATA
labels = df["label"].values
train_df, temp_df = train_test_split(df, test_size=0.25, random_state=42, stratify=labels)
val_df, test_df   = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

train_dataset = Dataset.from_pandas(train_df[["content","label"]])
val_dataset   = Dataset.from_pandas(val_df[["content","label"]])
test_dataset  = Dataset.from_pandas(test_df[["content","label"]])

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

# 8. TOKENIZER
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

tokenized = dataset_dict.map(tokenize_fn, batched=True, remove_columns=["content"])
tokenized = tokenized.rename_column("label", "labels")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 9. LOAD DistilBERT + PEFT LoRA

base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_lin", "k_lin", "v_lin"]   # DistilBERT attention layers
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# 10. CLASS WEIGHTS
class_weights = compute_class_weight("balanced", classes=[0,1], y=df["label"])
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# 11. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted", zero_division=0)

    acc = accuracy_score(labels, preds)

    probs = torch.softmax(torch.tensor(logits), dim=1)[:,1].numpy()
    auc = roc_auc_score(labels, probs)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc
    }

# 12. WEIGHTED TRAINER
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 13. TRAINING ARGS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# 14. TRAINER INIT
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# 15. TRAIN
print("\n==============================")
print("🚀 TRAINING DistilBERT + LoRA")
print("==============================\n")
trainer.train()

# 16. EVALUATE
print("\n🎯 TEST RESULTS")
results = trainer.evaluate(tokenized["test"])
for k,v in results.items():
    print(f"{k}: {v}")

# 17. SAVE FINAL MODEL
final_model_path = os.path.join(OUTPUT_DIR, "final_best_model")
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print("\n✅ Saved model to:", final_model_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.0 MB/s eta 0:00:00
Device: CPU | CUDA: False | VRAM: 0.0 GB


ValueError: mount failed